<a href="https://colab.research.google.com/github/s1250103/Public_mori-lab/blob/preprcing-learning/ex2(imitation).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# setting

In [1]:
%tensorflow_version 2.x

In [2]:
!pip install -q keras_tuner

     |████████████████████████████████| 71kB 3.6MB/s 


In [3]:
import numpy as np
import shutil
  
import tensorflow as tf
from tensorflow.keras import layers, models, initializers, callbacks
from tensorflow.keras.datasets import mnist

from pandas import DataFrame
import matplotlib as mpl
import matplotlib.pyplot as plt

np.random.seed(20190222)
tf.random.set_seed(20190222)

# model creating and hyper prametar designing

In [4]:
from tensorflow import keras
from tensorflow.keras.layers import Dense

def build_model(hp):
    model = models.Sequential()
    model.add(layers.Reshape((28, 28, 1), input_shape=(28*28,), name='reshape'))

    # drop1

    ## tune
    # model.add(layers.Dropout(hp.Choice('drop1_rate', values=[0.001, 0.5])))
    model.add(layers.Dropout(0.5))

    ##before
    model.add(layers.Conv2D(16, (5, 5), padding='same',
                        kernel_initializer=initializers.TruncatedNormal(),
                        use_bias=True, activation='relu',
                        name='conv_filter'))

    ##tune
    # model.add(layers.Conv2D(hp.Int('conv1_units',
    #                                min_value=8,
    #                                max_value=64,
    #                                step=8), (5, 5), padding='same',
    #                     kernel_initializer=initializers.TruncatedNormal(),
    #                     use_bias=True, activation='relu',
    #                     name='conv_filter'))
    
    

    model.add(layers.MaxPooling2D((2, 2), name='max_pooling'))
    model.add(layers.Flatten(name='flatten'))
    #before
    model.add(layers.Dense(1024, activation='relu',
                          kernel_initializer=initializers.TruncatedNormal(),
                          name='hidden'))
    # # tune
    # model.add(
    #     layers.Dense(units=hp.Int('units',
    #                               min_value=512,
    #                               max_value=1024,
    #                               step=512),
    #                  activation='relu',
    #                 kernel_initializer=initializers.TruncatedNormal(),
    #                 name='hidden'))

    model.add(layers.Dropout(hp.Choice('drop_rate', values=[0.001, 0.5])))

    model.add(layers.Dense(10, activation='softmax', name='softmax'))


    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-3, 1e-4])),
              loss='categorical_crossentropy',
              metrics=['acc'])
    
    
    return model

# tuner designing

In [9]:
from kerastuner.tuners import Hyperband

tuner = Hyperband(
    build_model,
    objective='val_acc',
    max_epochs=5,
    directory='my_dir',
    project_name='tf',
    overwrite=False
    )

INFO:tensorflow:Reloading Oracle from existing project my_dir/tf/oracle.json
INFO:tensorflow:Reloading Tuner from my_dir/tf/tuner0.json


In [6]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
drop_rate (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.001, 0.5], 'ordered': True}
learning_rate (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.001, 0.0001], 'ordered': True}


# gathering data

In [7]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape(
                  (len(train_images), 784)).astype('float32') / 255
test_images = test_images.reshape(
                  (len(test_images), 784)).astype('float32') / 255
train_labels = tf.keras.utils.to_categorical(train_labels, 10)
test_labels = tf.keras.utils.to_categorical(test_labels, 10)

11493376/11490434 [==============================] - 0s 0us/step


# searching param

In [8]:
%%time
tuner.search(train_images, train_labels,
             epochs=10,
             validation_data=(test_images, test_labels))

Trial 4 Complete [00h 00m 25s]
val_acc: 0.9595999717712402

Best val_acc So Far: 0.9789000153541565
Total elapsed time: 00h 01m 48s
INFO:tensorflow:Oracle triggered exit
CPU times: user 1min 21s, sys: 26.5 s, total: 1min 48s
Wall time: 1min 48s


# reloading tuner

In [21]:
# reload existing tuner
tuner = Hyperband(
    build_model,
    objective='val_loss',
    max_epochs=0,
    directory='my_dir',
    project_name='tf',
    overwrite=False
    )

INFO:tensorflow:Reloading Oracle from existing project my_dir/tf/oracle.json
INFO:tensorflow:Reloading Tuner from my_dir/tf/tuner0.json


# confirming the contents

In [22]:
tuner.results_summary()

Results summary
Results in my_dir/tf
Showing 10 best trials
Objective(name='val_loss', direction='min')
Trial summary
Hyperparameters:
drop1_rate: 0.001
learning_rate: 0.0001
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.9786999821662903
Trial summary
Hyperparameters:
drop1_rate: 0.5
learning_rate: 0.0001
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.9800999760627747
Trial summary
Hyperparameters:
drop1_rate: 0.001
learning_rate: 0.001
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.9860000014305115
Trial summary
Hyperparameters:
drop1_rate: 0.5
learning_rate: 0.001
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.9861999750137329
